In [23]:
#| default_exp 05_transfer_learning

In [1]:
import torch,torchvision
print(torch.__version__)
print(torchvision.__version__)

2.0.1
0.15.2


In [5]:
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

[INFO] Couldn't find going_modular scripts... downloading them from GitHub.
Cloning into 'pytorch-deep-learning'...
remote: Enumerating objects: 3842, done.
remote: Counting objects: 100% (485/485), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 3842 (delta 252), reused 404 (delta 206), pack-reused 3357
Receiving objects: 100% (3842/3842), 650.36 MiB | 30.17 MiB/s, done.
Resolving deltas: 100% (2208/2208), done.
Updating files: 100% (248/248), done.


In [33]:
#| export
import zipfile,requests,os
from pathlib import Path

def download_blob_git(url:str,file_name:str):
    """Downloads the blob or zip file from github rawlink. Only if file doesn't exist, if not simply exists.

    Args:
    url: Raw link need to be given, should have raw word in the link,rightclick raw option on github and copy link address.
    file_name: A name to be stored for that files in data folder should be same as github name

    Returns:
    None, Unzips the file and removes the zipped file. 
    """
    fpath=file_name
    image_path=Path(f'data/{fpath}')
    print(image_path)
    if image_path.is_file():
        print('Already exists skipping download')
    else:
        print('File doesnot exist downloading it')
        image_path.mkdir(parents=True,exist_ok=True)

        with open(f'data/{fpath}.zip','wb') as f:
            request=requests.get(url)
            print('Downloading the file')
            f.write(request.content)
        with zipfile.ZipFile(f'data/{fpath}.zip','r') as z:
            print('Unzipping the files')
            z.extractall(image_path)
        os.remove(f'data/{fpath}.zip')

In [34]:
download_blob_git('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip',"pizza_steak_sushi")

data/pizza_steak_sushi
File doesnot exist downloading it
Unzipping the files


In [2]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
manual_transforms=transforms.Compose([
                                        transforms.Resize((224,224)),
                                        transforms.ToTensor(),
                                        normalize
                                    ])

In [41]:
from pathlib import Path
train_dir = Path('data/pizza_steak_sushi/train')
test_dir = Path('data/pizza_steak_sushi/test')

In [42]:
test_dir

PosixPath('data/pizza_steak_sushi/test')

In [43]:
from going_modular.going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                           test_dir=test_dir,
                           transform=manual_transforms,
                           batch_size=32,
                           num_workers=4)
train_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x1346aaac0>,
 ['pizza', 'steak', 'sushi'])

In [50]:
for (x,y) in (train_dataloader):
    print(y.shape)
    break

torch.Size([32])
0


In [53]:
model_weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT
model_weights

EfficientNet_B0_Weights.IMAGENET1K_V1

In [55]:
auto_transform=model_weights.transforms()
auto_transform

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [56]:
from going_modular.going_modular import data_setup
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                           test_dir=test_dir,
                           transform=auto_transform,
                           batch_size=32,
                           num_workers=4)
train_dataloader,class_names

(<torch.utils.data.dataloader.DataLoader at 0x1360713a0>,
 ['pizza', 'steak', 'sushi'])

In [58]:
# Ways to get pre-trained models
#     1. pytorch library
#     2.timm torch image library
#     3.Hugging face
#     4.Paperswithcode

# To consider
#     1. Speed
#     2.Size
#     3.Performance
    

In [59]:
model=torchvision.models.efficientnet_b0(weights=model_weights)
model

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /Users/hemanththaluru/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████████████████████████████| 20.5M/20.5M [00:00<00:00, 27.6MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [62]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [63]:
import torchinfo

In [86]:
summary=torchinfo.summary(model=model,input_size=(1,3,224,224),
                         col_names=["input_size",
                                    "output_size",
                                    "num_params",
                                    "trainable",],
                          col_width=13,
                          row_settings=['var_names']
                         )
summary

Layer (type (var_name))                                      Input Shape   Output Shape  Param #       Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224] [1, 1000]     --            True
├─Sequential (features)                                      [1, 3, 224, 224] [1, 1280, 7, 7] --            True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224] [1, 32, 112, 112] --            True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224] [1, 32, 112, 112] 864           True
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112] [1, 32, 112, 112] 64            True
│    │    └─SiLU (2)                                         [1, 32, 112, 112] [1, 32, 112, 112] --            --
│    └─Sequential (1)                                        [1, 32, 112, 112] [1, 16, 112, 112] --            True
│    │    └─MBConv (0)                                       [1, 32, 112, 112] [1, 16, 

In [67]:
# Types of transfer Learning
#     1. Original Model -> no change at all -> good when training data is almost similar original train data
#     2. Feature extracted model -> change only classifier layer(last layer) -> good when less train data
#     3.Fine tuned model -> changing last few layer after feature extraction completed -> good when more train data

In [104]:
# Freezing sequential layers by keeping req_grad=false for this layers
for params in model.features.parameters():
    params.requires_grad=False

In [105]:
summary2=torchinfo.summary(model=model,input_size=(1,3,224,224),
                         col_names=["input_size",
                                    "output_size",
                                    "num_params",
                                    "trainable",],
                          col_width=13,
                          row_settings=['var_names']
                         )
summary2

Layer (type (var_name))                                      Input Shape   Output Shape  Param #       Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224] [1, 1000]     --            Partial
├─Sequential (features)                                      [1, 3, 224, 224] [1, 1280, 7, 7] --            False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224] [1, 32, 112, 112] --            False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224] [1, 32, 112, 112] (864)         False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112] [1, 32, 112, 112] (64)          False
│    │    └─SiLU (2)                                         [1, 32, 112, 112] [1, 32, 112, 112] --            --
│    └─Sequential (1)                                        [1, 32, 112, 112] [1, 16, 112, 112] --            False
│    │    └─MBConv (0)                                       [1, 32, 112, 112] 

In [111]:
from torch import nn

In [112]:
model.classifier=nn.Sequential(
    nn.Dropout(p=0.2,inplace=True),
    nn.Linear(in_features=1280,out_features=3)
)

In [113]:
summary3=torchinfo.summary(model=model,input_size=(1,3,224,224),
                         col_names=["input_size",
                                    "output_size",
                                    "num_params",
                                    "trainable",],
                          col_width=13,
                          row_settings=['var_names']
                         )
summary3

Layer (type (var_name))                                      Input Shape   Output Shape  Param #       Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224] [1, 3]        --            Partial
├─Sequential (features)                                      [1, 3, 224, 224] [1, 1280, 7, 7] --            False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224] [1, 32, 112, 112] --            False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224] [1, 32, 112, 112] (864)         False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112] [1, 32, 112, 112] (64)          False
│    │    └─SiLU (2)                                         [1, 32, 112, 112] [1, 32, 112, 112] --            --
│    └─Sequential (1)                                        [1, 32, 112, 112] [1, 16, 112, 112] --            False
│    │    └─MBConv (0)                                       [1, 32, 112, 112] 

In [114]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)

In [118]:
#Training the model
from going_modular.going_modular import engine
from timeit import default_timer as timer

In [119]:

start_timer=timer()
results=engine.train(model=model,train_dataloader=train_dataloader,test_dataloader=test_dataloader,
                    optimizer=optimizer,loss_fn=loss_fn,epochs=5,device='cpu')
print(f'Total time:{-start_timer+timer()}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0573 | train_acc: 0.4258 | test_loss: 0.8608 | test_acc: 0.7737
Epoch: 2 | train_loss: 0.8585 | train_acc: 0.7734 | test_loss: 0.7128 | test_acc: 0.8854
Epoch: 3 | train_loss: 0.8476 | train_acc: 0.6680 | test_loss: 0.6350 | test_acc: 0.8646
Epoch: 4 | train_loss: 0.6742 | train_acc: 0.8867 | test_loss: 0.6458 | test_acc: 0.8352
Epoch: 5 | train_loss: 0.5969 | train_acc: 0.9102 | test_loss: 0.6843 | test_acc: 0.7841
Total time:381.91801533299986
